# Numpyro Jax PlayGround

My starting notebook where I install all of the necessary libraries and load some easy 1D/2D Regression data to play around with.

In [ ]:
# @title Install
try:
    import sys, os
    from pyprojroot import here

    # spyder up to find the root
    root = here(project_files=[".root"])

    # append to path
    sys.path.append(str(root))

except ModuleNotFoundError:
    pass
    # #@title Install Packages
    # %%capture
    # !pip install pyprojroot jax jaxlib chex numpyro flax distrax gpjax numpy pandas seaborn matplotlib corner loguru nb_black sklearn

In [ ]:
# @title Load Packages
# TYPE HINTS
from typing import Tuple, Optional, Dict, Callable, Union

# JAX SETTINGS
import jax
import jax.numpy as jnp
import jax.random as random
from jax.config import config

config.update("jax_enable_x64", True)


# JAX UTILITY LIBRARIES
import chex

# NUMPYRO SETTINGS
import numpyro
import numpyro.distributions as dist
from numpyro.infer.autoguide import AutoDiagonalNormal
from numpyro.infer import SVI, Trace_ELBO

# NUMPY SETTINGS
import numpy as np

np.set_printoptions(precision=3, suppress=True)

# MATPLOTLIB Settings
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# SEABORN SETTINGS
import seaborn as sns

sns.set_context(context="poster", font_scale=0.8)

# PANDAS SETTINGS
import pandas as pd

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

# LOGGING SETTINGS
import loguru

%load_ext lab_black
%load_ext autoreload
%autoreload 2

## Demo Data

In [ ]:
from src.data import regression_near_square

(
    Xtrain,
    Xtrain_noise,
    ytrain,
    xtest,
    xtest_noise,
    ytest,
    ytest_noise,
) = regression_near_square(
    n_train=60, n_test=1_000, x_noise=0.5, y_noise=0.01, seed=123, buffer=0.3
)

fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(Xtrain, ytrain, color="tab:orange", label="Data")
ax.plot(xtest, ytest, color="black", label="True Function")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(Xtrain_noise, ytrain, color="tab:orange", label="Data")
ax.plot(xtest, ytest, color="black", label="True Function")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(xtest_noise, ytest_noise, color="tab:orange", label="Test Data")
ax.plot(xtest, ytest, color="black", label="True Function")
ax.legend()
plt.tight_layout()
plt.show()

## Standard GP Model

In [ ]:
from chex import Array
from src.utils import identity_matrix, add_to_diagonal
from src.kernels import RBF

rbf_kernel = RBF(length_scale=1.0, variance=1.0)
K = rbf_kernel.gram(Xtrain)

# check shape
chex.assert_shape(K, (Xtrain.shape[0], Xtrain.shape[0]))

### Model

In [ ]:
from src.exact import GPRModel
from src.means import zero_mean

jitter = 1e-5


def numpyro_model(X, y):

    #     # Set priors on hyperparameters.
    #     η = numpyro.sample("variance", dist.HalfCauchy(scale=5.0))
    #     ℓ = numpyro.sample("length_scale", dist.Gamma(2.0, 1.0))
    #     σ = numpyro.sample("obs_noise", dist.HalfCauchy(scale=5.0))

    # set params and constraints on hyperparams
    η = numpyro.param("variance", init_value=1.0, constraints=dist.constraints.positive)
    ℓ = numpyro.param(
        "length_scale", init_value=1.0, constraints=dist.constraints.positive
    )
    σ = numpyro.param(
        "obs_noise", init_value=0.01, onstraints=dist.constraints.positive
    )

    # Kernel Function
    rbf_kernel = RBF(variance=η, length_scale=ℓ)

    # GP Model
    gp_model = GPRModel(
        X=X, y=y, mean=zero_mean, kernel=rbf_kernel, obs_noise=σ, jitter=jitter
    )

    # Sample y according SGP
    return gp_model.to_numpyro(y=y)


def empty_guide(X, y):
    pass

In [ ]:
with numpyro.handlers.seed(rng_seed=123):
    t = numpyro_model(Xtrain, ytrain)

assert t.shape == Xtrain.shape

In [ ]:
from numpyro.infer.autoguide import AutoDelta

# ===================
# Model
# ===================
# GP model

# delta guide - basically deterministic
delta_guide = AutoDelta(numpyro_model)
mll_guide = empty_guide

In [ ]:
# reproducibility
rng_key = random.PRNGKey(0)


# Setup
optimizer = numpyro.optim.Adam(step_size=0.01)
# optimizer = numpyro.optim.Minimize()
# optimizer = optax.adamw(learning_rate=0.1)
svi = SVI(numpyro_model, mll_guide, optimizer, loss=Trace_ELBO())
svi_results = svi.run(random.PRNGKey(1), 100, Xtrain, ytrain.T)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(svi_results.losses)
ax.set(title="Loss", xlabel="Iterations", ylabel="Negative Log-Likelihood")
plt.show()

In [ ]:
from pprint import pprint

# Take them directly
learned_params = svi_results.params
# x_u = learned_params["x_u"]
# learned_params = delta_guide.median(learned_params)
# learned_params["x_u"] = x_u
learned_params

## Predictions

### Cleanr

In [ ]:
from src.exact import init_gp_predictive, get_cond_params

gp_pred = init_gp_predictive(RBF, learned_params, Xtrain, ytrain, jitter=1e-5)

mu, var = gp_pred.predict_y(xtest)

std = jnp.sqrt(var.squeeze())

In [ ]:
ci = 1.96
one_stddev = ci * std

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(Xtrain, ytrain.squeeze(), "o", color="tab:orange", label="Training Data")
ax.plot(xtest, ytest, color="black", linestyle="-", label="True Function")
ax.plot(
    xtest,
    mu.ravel(),
    color="Blue",
    linestyle="--",
    linewidth=3,
    label="Predictive Mean",
)
ax.fill_between(
    xtest.ravel(),
    mu.ravel() - one_stddev,
    mu.ravel() + one_stddev,
    alpha=0.4,
    color="tab:blue",
    label=f" 95% Confidence Interval",
)
ax.plot(xtest, mu.ravel() - one_stddev, linestyle="--", color="tab:blue")
ax.plot(xtest, mu.ravel() + one_stddev, linestyle="--", color="tab:blue")
plt.tight_layout()
plt.legend(fontsize=12)
plt.show()

#### Calibration

In [ ]:
from uncertainty_toolbox import viz as utviz

In [ ]:
mu, var = gp_pred.predict_y(Xtrain)

std = jnp.sqrt(var.squeeze())

utviz.plot_calibration(y_pred=mu.ravel(), y_std=std.ravel(), y_true=ytrain.ravel())

In [ ]:
mu, var = gp_pred.predict_y(xtest)

std = jnp.sqrt(var.squeeze())

utviz.plot_calibration(y_pred=mu.ravel(), y_std=std.ravel(), y_true=ytest.ravel())

### Noisy

In [ ]:
from src.exact import init_gp_predictive, get_cond_params

gp_pred = init_gp_predictive(RBF, learned_params, Xtrain, ytrain, jitter=1e-8)

mu, var = gp_pred.predict_y(xtest_noise)

std = jnp.sqrt(var.squeeze())

ci = 1.96
one_stddev = ci * std

In [ ]:
idx_samples = np.random.choice(np.arange(xtest_noise.shape[0]), 50)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(
    xtest_noise, ytest_noise, marker="o", s=30, color="tab:orange", label="Testing Data"
)
ax.plot(xtest, ytest, color="black", linestyle="-", label="True Function")
ax.plot(
    xtest_noise,
    mu.ravel(),
    color="Blue",
    linestyle="--",
    linewidth=3,
    label="Predictive Mean",
)
ax.fill_between(
    xtest_noise.ravel(),
    mu.ravel() - one_stddev,
    mu.ravel() + one_stddev,
    alpha=0.4,
    color="tab:blue",
    label=f" 95% Confidence Interval",
)
ax.plot(xtest_noise, mu.ravel() - one_stddev, linestyle="--", color="tab:blue")
ax.plot(xtest_noise, mu.ravel() + one_stddev, linestyle="--", color="tab:blue")
plt.tight_layout()
plt.legend(fontsize=12)
plt.show()

In [ ]:
mu, var = gp_pred.predict_y(xtest_noise)

std = jnp.sqrt(var.squeeze())

utviz.plot_calibration(y_pred=mu.ravel(), y_std=std.ravel(), y_true=ytest_noise.ravel())

### Monte Carlo Sampling

In this first example, we will approximate the actual posterior using Monte Carlo sampling. Using the following formula:

$$
p(f_*|\mu_\mathbf{x}, \Sigma_\mathbf{x}, \mathcal{D}) \approx \frac{1}{T}\sum_{t=1}^T \mathcal{N}\left(f_*|\mu_\mathcal{GP}(\mathbf{x}_*^t),\sigma^2_\mathcal{GP}(\mathbf{x}_*^t) \right) 
$$

This will approach the real posterior as $T$ grows. What we want to demonstrate is that this posterior is non-Gaussian when our inputs are non-Gaussian.

In this example, we assume that $\mathbf{x}_* \sim \mathcal{N}(\mathbf{\mu_x}, \mathbf{\Sigma_x})$. We're assuming our data points come We can reparameterize this as f


we will be using the following form to do the sampling:

$$
\mathbf{x}_T = \mu_\mathbf{x} + \mathbf{L}\mathbf{z}
$$

where $\mathbf{z}\sim \mathcal{N}(\mathbf{0}, \mathbf{I})$. So we will draw 10,000 samples and then propagate all of these through the non-linear function.

In [ ]:
from src.uncertain.monte_carlo import init_mc_transform


egp_pred_mc = init_mc_transform(gp_pred, n_features=1, n_samples=1_000)
x_cov = jnp.array([0.5]).reshape(-1, 1) ** 2
key = jax.random.PRNGKey(123)

mu_mc, var_mc = egp_pred_mc.predict_f(key, xtest_noise, x_cov, False)
std_mc = jnp.sqrt(var_mc).ravel()
mu_mc.shape, std_mc.shape

In [ ]:
pred_jitted = jax.jit(jax.partial(egp_pred_mc.predict_f, full_covariance=False))
mu_mc, var_mc = pred_jitted(key, xtest_noise, x_cov)
std_mc = jnp.sqrt(var_mc).ravel()
mu_mc.shape, std_mc.shape

In [ ]:
%timeit egp_pred_mc.predict_f(key, xtest_noise.block_until_ready(), x_cov.block_until_ready(), False)
%timeit pred_jitted(key, xtest_noise.block_until_ready(), x_cov.block_until_ready())

In [ ]:
ci = 1.96
one_stddev = ci * std_mc

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(
    xtest_noise, ytest_noise, marker="o", s=30, color="tab:orange", label="Testing Data"
)
ax.plot(xtest, ytest, color="black", linestyle="-", label="True Function")
ax.plot(
    xtest_noise,
    mu.ravel(),
    color="Blue",
    linestyle="--",
    linewidth=3,
    label="Predictive Mean",
)
ax.fill_between(
    xtest_noise.ravel(),
    mu.ravel() - one_stddev,
    mu.ravel() + one_stddev,
    alpha=0.4,
    color="tab:blue",
    label=f" 95% Confidence Interval",
)
ax.plot(xtest_noise, mu.ravel() - one_stddev, linestyle="--", color="tab:blue")
ax.plot(xtest_noise, mu.ravel() + one_stddev, linestyle="--", color="tab:blue")
plt.tight_layout()
plt.legend(fontsize=12)
plt.show()

In [ ]:
utviz.plot_calibration(
    y_pred=mu_mc.ravel(), y_std=std_mc.ravel(), y_true=ytest_noise.ravel()
)

### Unscented Transformation

In [ ]:
from src.uncertain.unscented import init_unscented_transform


egp_pred_unc = init_unscented_transform(gp_pred, n_features=1)
x_cov = jnp.array([0.5]).reshape(-1, 1) ** 2
key = jax.random.PRNGKey(123)
mu_unc = egp_pred_unc.predict_mean(xtest_noise, x_cov)
cov_unc = egp_pred_unc.predict_cov(xtest_noise, x_cov)
std_unc = jnp.sqrt(jnp.diag(cov_unc.squeeze()))
mu_unc, var_unc = egp_pred_unc.predict_f(xtest_noise, x_cov, False)
std_unc_2 = jnp.sqrt(var_unc.squeeze())

In [ ]:
plt.plot(xtest_noise, mu_mc)
plt.plot(xtest_noise, mu_unc)
plt.plot(xtest_noise, mu)

In [ ]:
ci = 1.96
one_stddev = ci * std_unc_2

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(
    xtest_noise, ytest_noise, marker="o", s=30, color="tab:orange", label="Testing Data"
)
ax.plot(xtest, ytest, color="black", linestyle="-", label="True Function")
ax.plot(
    xtest_noise,
    mu.ravel(),
    color="Blue",
    linestyle="--",
    linewidth=3,
    label="Predictive Mean",
)
ax.fill_between(
    xtest_noise.ravel(),
    mu.ravel() - one_stddev,
    mu.ravel() + one_stddev,
    alpha=0.4,
    color="tab:blue",
    label=f" 95% Confidence Interval",
)
ax.plot(xtest_noise, mu.ravel() - one_stddev, linestyle="--", color="tab:blue")
ax.plot(xtest_noise, mu.ravel() + one_stddev, linestyle="--", color="tab:blue")
plt.tight_layout()
plt.legend(fontsize=12)
plt.show()

In [ ]:
utviz.plot_calibration(
    y_pred=mu_unc.ravel(), y_std=std_unc.ravel(), y_true=ytest_noise.ravel()
)

In [ ]:
from gpjax.gps import Prior
from gpjax.mean_functions import Zero
from gpjax.kernels import RBF
from gpjax.likelihoods import Gaussian
from gpjax.types import Dataset


# GP Prior
mean_function = Zero()
kernel = RBF()
prior = Prior(mean_function=mean_function, kernel=kernel)

# GP Likelihood
lik = Gaussian()

# GP Posterior
posterior = prior * lik

# initialize training dataset
training_ds = Dataset(X=X, y=y_noise)

#### Parameters

In [ ]:
from gpjax.parameters import initialise
import numpyro.distributions as dist
from gpjax.interfaces.numpyro import numpyro_dict_params, add_constraints


# initialize parameters
params = initialise(posterior)

hyperpriors = {
    "lengthscale": dist.LogNormal(0.0, 10.0),
    "variance": dist.LogNormal(0.0, 10.0),
    "obs_noise": dist.LogNormal(0.0, 10.0),
}

hyperpriors = {
    "lengthscale": 1.0,
    "variance": 1.0,
    "obs_noise": 0.01,
}


# convert to numpyro-style params
numpyro_params = numpyro_dict_params(hyperpriors)

# convert to numpyro-style params
numpyro_params = add_constraints(numpyro_params, dist.constraints.softplus_positive)

### Inference (SVI)

In [ ]:
from numpyro.infer.autoguide import AutoDelta
from gpjax.interfaces.numpyro import numpyro_marginal_ll, numpyro_dict_params


# initialize numpyro-style GP model
npy_model = numpyro_marginal_ll(posterior, numpyro_params)

# approximate posterior
guide = AutoDelta(npy_model)

### Training

In [ ]:
from numpyro.infer import SVI, Trace_ELBO

# reproducibility
rng_key = jax.random.PRNGKey(0)
n_iterations = 2_000


# numpyro specific optimizer
optimizer = numpyro.optim.Adam(step_size=0.005)

# stochastic variational inference (pseudo)
svi = SVI(npy_model, guide, optimizer, loss=Trace_ELBO())
svi_results = svi.run(jax.random.PRNGKey(1), n_iterations, training_ds)

#### Losses

In [ ]:
fig, ax = plt.subplots()

ax.plot(svi_results.losses)
ax.set(title="Loss", xlabel="Iterations", ylabel="ELBO")
plt.show()

# Take them directly


In [ ]:
learned_params = svi_results.params
learned_params

In [ ]:
from gpjax import mean, variance

meanf = mean(posterior, learned_params, training_ds)
covarf = variance(posterior, learned_params, training_ds)
varf = lambda x: jnp.atleast_1d(jnp.diag(covarf(x)))


mu = meanf(Xtest).squeeze()
cov = covarf(Xtest).squeeze()
var = varf(Xtest).squeeze()

one_stddev = 1.96 * jnp.sqrt(jnp.diag(cov))

In [ ]:
def get_sorted_indices(X):
    return jnp.argsort(X, axis=0).squeeze()


def apply_sorted_indices(X, sorted_idx):
    return X[(sorted_idx,)]

In [ ]:
def plot_1D_GP_clean(Xtest, y_mu, y_var):

    one_stddev = 1.96 * jnp.sqrt(y_var)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(X.ravel(), y.squeeze(), "o", color="tab:orange")

    ax.plot(Xtest.ravel(), y_mu, color="tab:blue")
    ax.fill_between(
        Xtest.ravel(),
        y_mu.ravel() - one_stddev.ravel(),
        y_mu.ravel() + one_stddev.ravel(),
        alpha=0.4,
        color="tab:blue",
    )
    ax.plot(Xtest.ravel(), y_mu.ravel() - one_stddev, linestyle="--", color="tab:blue")
    ax.plot(Xtest.ravel(), y_mu.ravel() + one_stddev, linestyle="--", color="tab:blue")
    plt.show()
    return fig, ax

In [ ]:
plot_1D_GP_clean(Xtest, mu, jnp.diag(cov))

### Predictions on Noisy Data

In [ ]:
x_noise = 0.5
input_cov = jnp.array([x_noise]).reshape(-1, 1) ** 2

ntest = 100


Xtest = np.linspace(-10.1, 10.1, ntest)[:, None]
ytest = f(Xtest)

demo_sample_idx = 47

key, xt_rng = jax.random.split(key, 2)

Xtest_noisy = Xtest + x_noise * jax.random.normal(xt_rng, shape=Xtest.shape)


idx_sorted = jnp.argsort(Xtest_noisy, axis=0)

# Xtest = Xtest[(idx_sorted,)]
Xtest_noisy = Xtest_noisy[(idx_sorted,)][..., 0]
ytest_noisy = ytest[(idx_sorted,)]

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
ax[1].scatter(Xtest_noisy, ytest_noisy, color="red")
ax[0].scatter(Xtest, ytest, color="red")
ax[1].scatter(
    Xtest_noisy[demo_sample_idx],
    ytest_noisy[demo_sample_idx],
    marker=".",
    s=300,
    color="black",
)

plt.show()

In [ ]:
def plot_1D_GP_noisy(Xtest_noisy, y_mu, y_var):

    one_stddev = 1.96 * jnp.sqrt(y_var)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(Xtest_noisy.squeeze(), ytest_noisy.squeeze(), "o", color="tab:orange")

    ax.plot(Xtest_noisy.squeeze(), y_mu.squeeze(), color="tab:blue")
    ax.fill_between(
        Xtest_noisy.squeeze(),
        y_mu.squeeze() - one_stddev.squeeze(),
        y_mu.ravel() + one_stddev.squeeze(),
        alpha=0.4,
        color="tab:blue",
    )
    ax.plot(
        Xtest_noisy.squeeze(),
        y_mu.squeeze() - one_stddev.squeeze(),
        linestyle="--",
        color="tab:blue",
    )
    ax.plot(
        Xtest_noisy.squeeze(),
        y_mu.squeeze() + one_stddev.squeeze(),
        linestyle="--",
        color="tab:blue",
    )
    plt.show()
    return fig, ax

In [ ]:
mu = meanf(Xtest_noisy).squeeze()
var = varf(Xtest_noisy).squeeze()


one_stddev = 1.96 * jnp.sqrt(var)

In [ ]:
plot_1D_GP_noisy(Xtest_noisy, mu, var)

## Noisy Test Samples

In Gaussian processes, the original formulation dictates that we assume there is some noise in the observations, $y$ and that we observe the real inputs $\mathbf{x}$. So we'll see that this it is not trivial to modify this formulation to account for uncertain inputs. Let's assume that we have a data set $\mathcal{D}=\{\mathbf{X}, \boldsymbol{y} \}$. In this case we assume the following relationship between our inputs, $\mathbf{x}$, and outputs, $y$:

$$
\begin{aligned}
y_n &= f(\mathbf{x}_n)+ \epsilon_y \\
\epsilon_y &\sim \mathcal{N}(0,\sigma_y^2)
\end{aligned}
$$

Let's also assume that we have a standard GP model optimized and fitted to this data set. We're not assuming noisy inputs during the training phase so we will use the standard log-likelihood maximization procedure. However, during the testing phase, we will assume that our inputs are noisy. For simplicity, we can assume our test data set is normally distributed with a mean $\mu_\mathbf{x}$ and variance $\Sigma_\mathbf{x}$. So we will have:

$$
\mathbf{x}_* \sim \mathcal{N}(\mu_\mathbf{x}, \Sigma_\mathbf{x}) $$

or equivalently we can reparameterize it like so:

$$
\begin{aligned}
\mathbf{x}_* &=\mu_\mathbf{x}+ \epsilon_\mathbf{x} \\
\epsilon_\mathbf{x} &\sim \mathcal{N}(0, \Sigma_\mathbf{x})
\end{aligned}
$$

If we consider the predictive distribution given by $p(f_*|\mathbf{x}_*, \mathcal{D})$,  we need to marginalize out the input distribution. So the full integral appears as follows.

$$p(f_*|\mu_\mathbf{x}, \Sigma_\mathbf{x},\mathcal{D}) = \int p(f_*|\mathbf{x}_*,\mathcal{D})\;\mathcal{N}(\mathbf{x}_*|\mu_\mathbf{x},\Sigma_\mathbf{x})\; d\mathbf{x}_*$$

If we use the GP formulation, we have a closed-form deterministic predictive distribution for $p(f_*|\mathbf{x}_*,\mathcal{D})$. Plugging this into the above equation gives us:

$$p(f_*|\mu_\mathbf{x},\Sigma_\mathbf{x},\mathcal{D}) = \int \mathcal{N}\left(f_*|\mu_\mathcal{GP}(\mathbf{x}_*),\sigma^2_\mathcal{GP}(\mathbf{x}_*) \right) \; \mathcal{N}(\mathbf{x}_*|\mu_\mathbf{x}, \Sigma_\mathbf{x})\; d\mathbf{x}_*$$

So this integral is intractable because if we consider the terms within the GP predictive mean and predictive variance, we will need to calculate the integral of an inverse kernel function, $\mathbf{K}_\mathcal{GP}^{-1}$. Below we outline some of the most popular methods found in the literature.

### Demo - Single Point

For to demonstrate the function, we're going to do the steps assuming no batch dimension. In other words, we're working with a test vector $\mathbf{x}_* \in \mathbb{R}^{D}$, **not** a matrix of samples $\mathbf{X}_* \in \mathbb{R}^{N\times D}$.

In [ ]:
xtest_noisy = Xtest_noisy[demo_sample_idx]
mu_demo = {}

mu_demo = {"standard": meanf(xtest_noisy)}
covar_demo = {"standard": varf(xtest_noisy)}

In [ ]:
plt.scatter(Xtest_noisy, ytest_noisy, s=10, color="tab:orange")
plt.scatter(xtest_noisy, mu_demo["standard"], marker=".", s=300, color="black")
plt.axhline(mu_demo["standard"], linestyle="--", color="black")

In [ ]:
# x_mu = np.array([-0.2])
# x_cov = np.tile(np.diag(np.array([3.0])), (1, 1))

# # get function
# f = jax.vmap(jax.partial(predictive_mean, model))
# df = jax.vmap(jax.grad(jax.partial(predictive_mean, model)))

In [ ]:
# mu = f(Xtest)

# # plt.hist(x_mc_samples.squeeze(), bins=100, density=True);
# fig, ax = plt.subplots(figsize=(7, 5))
# ax.plot(Xtest, mu, color='red', linewidth=4, label=r"$f(x)$")
# ax.axvline(x_sample, linestyle="--", color='black', linewidth=4)
# ax.set(xticklabels="", xlabel="", yticklabels="")
# plt.fill_between(Xtest.squeeze(), lb, ub, label='CI', color='darkorange', alpha=0.2)
# ax.set(xlim=(-10, 10), ylim=(-2.1, 2.1))
# plt.scatter(X, y, label='Training', color='blue', s=60, zorder=3)
# plt.legend(fontsize=15)
# plt.show()

### Monte Carlo Sampling

In this first example, we will approximate the actual posterior using Monte Carlo sampling. Using the following formula:

$$
p(f_*|\mu_\mathbf{x}, \Sigma_\mathbf{x}, \mathcal{D}) \approx \frac{1}{T}\sum_{t=1}^T \mathcal{N}\left(f_*|\mu_\mathcal{GP}(\mathbf{x}_*^t),\sigma^2_\mathcal{GP}(\mathbf{x}_*^t) \right) 
$$

This will approach the real posterior as $T$ grows. What we want to demonstrate is that this posterior is non-Gaussian when our inputs are non-Gaussian.

In this example, we assume that $\mathbf{x}_* \sim \mathcal{N}(\mathbf{\mu_x}, \mathbf{\Sigma_x})$. We're assuming our data points come We can reparameterize this as f


we will be using the following form to do the sampling:

$$
\mathbf{x}_T = \mu_\mathbf{x} + \mathbf{L}\mathbf{z}
$$

where $\mathbf{z}\sim \mathcal{N}(\mathbf{0}, \mathbf{I})$. So we will draw 10,000 samples and then propagate all of these through the non-linear function.

In [ ]:
import tensorflow_probability.substrates.jax as tfp

dist = tfp.distributions

n_features = 1

z = dist.Normal(loc=np.zeros((n_features,)), scale=np.ones(n_features))

In [ ]:
# input covariance
L = jnp.linalg.cholesky(input_cov)

chex.assert_shape(
    L,
    (
        n_features,
        n_features,
    ),
)

# generate sigma points
n_mc_points = 100
sigma_pts = z.sample((n_mc_points,), key).T

chex.assert_shape(
    sigma_pts,
    (
        n_features,
        n_mc_points,
    ),
)

# calculate sigma points
x_mc_samples = xtest_noisy + L @ sigma_pts

chex.assert_shape(x_mc_samples, (n_features, n_mc_points))

# mean function predictions

f = lambda x: jnp.atleast_1d(meanf(x).squeeze())

y_mu_mc = jax.vmap(f, in_axes=2)(x_mc_samples[:, None, :])

y_mu_mc = jnp.atleast_1d(y_mu_mc)

chex.assert_shape(
    y_mu_mc,
    (
        n_mc_points,
        n_features,
    ),
)


# calculate mean of MC samples
wm = 1.0 / n_mc_points
y_mu = jnp.mean(y_mu_mc, axis=0)

chex.assert_shape(y_mu, (n_features,))

# calculate covariance of MC samples
wc = 1.0 / (n_mc_points - 1.0)
dfx = y_mu_mc - y_mu

chex.assert_shape(
    dfx,
    (
        n_mc_points,
        n_features,
    ),
)

x_cov = wc * dfx.T @ dfx

chex.assert_shape(
    x_cov,
    (
        n_features,
        n_features,
    ),
)

# input-output covariance
x_covx = wc * (x_mc_samples - xtest_noisy) @ dfx

chex.assert_shape(
    x_cov,
    (
        n_features,
        n_features,
    ),
)

In [ ]:
mu_demo["mc"] = y_mu

In [ ]:
# plt.hist(x_mc_samples.squeeze(), bins=100, density=True);
fig, ax = plt.subplots(figsize=(7, 1))
sns.kdeplot(
    ax=ax,
    x=x_mc_samples.squeeze(),
    bw_adjust=3.0,
    color="black",
    linewidth=4,
    fill=True,
    label=r"$p(x)$",
)
ax.axvline(xtest_noisy, linestyle="--", color="black", linewidth=4)
ax.set(yticklabels="", ylabel="", xticklabels="", xlim=(-10, 10))
# ax.set_ylim(-1.5, 1.1)
plt.legend(fontsize=15)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
sns.kdeplot(ax=ax, y=y_mu_mc.ravel(), bw_adjust=3.0, linewidth=4, label="MC Samples")
ax.set(xlabel="", xticklabels="", ylim=(-2.1, 2.1), yticklabels="")
ax.axhline(mu_demo["standard"], color="black", linestyle="--", label="Mean Prediction")
plt.legend()
plt.show()

#### Vectorized Over All Samples

In [ ]:
from typing import Callable


def init_mc_moment_transform(
    meanf: Callable, n_features: int, mc_samples: int = 100, covariance: bool = False
) -> Callable:

    f = lambda x: jnp.atleast_1d(meanf(x).squeeze())
    z = dist.Normal(loc=np.zeros((n_features,)), scale=np.ones(n_features))
    wm = 1.0 / mc_samples
    wc = 1.0 / (mc_samples - 1.0)

    def apply_transform(rng_key, x, x_cov):

        # sigma points
        sigma_pts = z.sample((mc_samples,), key)

        # cholesky for input covariance
        L = jnp.linalg.cholesky(input_cov)

        # calculate sigma points
        x_mc_samples = x + L @ sigma_pts.T

        # ===================
        # Mean
        # ===================

        # function predictions over mc samples
        y_mu_mc = jax.vmap(f, in_axes=1)(x_mc_samples).reshape(
            (
                mc_samples,
                n_features,
            )
        )

        # mean of mc samples
        y_mu = jnp.mean(y_mu_mc, axis=0)

        y_mu = jnp.atleast_1d(y_mu)

        # ===================
        # Covariance
        # ===================
        dfydx = y_mu_mc - y_mu

        y_cov = wc * dfydx.T @ dfydx

        if not covariance:
            y_cov = jnp.diag(y_cov)

            y_cov = jnp.atleast_1d(y_cov)

        return y_mu, y_cov

    return apply_transform

In [ ]:
# init function
n_features = 1
mc_samples = 1_000
mc_transform = init_mc_moment_transform(
    meanf, n_features=n_features, mc_samples=mc_samples
)

# apply function
xtest_noisy = Xtest_noisy[demo_sample_idx][..., 0]
input_cov = jnp.array([x_noise]).reshape(-1, 1)

x_mu, x_var = mc_transform(key, xtest_noisy, input_cov)

chex.assert_equal_shape([x_mu, x_var])
chex.assert_shape(x_mu, (n_features,))

In [ ]:
mc_transform_vectorized = jax.jit(jax.vmap(mc_transform, in_axes=(None, 0, None)))

In [ ]:
y_mu, y_var = mc_transform_vectorized(key, Xtest_noisy, input_cov)

chex.assert_equal_shape([y_mu, y_var])
chex.assert_shape(
    y_mu,
    (
        ntest,
        n_features,
    ),
)

y_std = 1.96 * jnp.sqrt(y_var)

In [ ]:
plot_1D_GP_noisy(Xtest_noisy, y_mu, y_var);

As you can see, this is a non-Gaussian distribution. Below we will see how we can approximate this as a Gaussian.

## Taylor Approximation

The 




### 1st Order Approximation

$$
\begin{aligned}
\tilde{\mathbf{\mu}}_\text{LinGP}(\mathbf{x_*})
&= \mathbf{\mu}_\text{GP}(\mathbf{\mu}_\mathbf{x_*})\\
\tilde{\mathbf{\Sigma}}^2_\text{LinGP} (\mathbf{x_*}) &= 
\mathbf{\Sigma}^2_\text{GP}(\mathbf{\mu}_\mathbf{x_*}) + 
\underbrace{\frac{\partial \mathbf{\mu}_\text{GP}(\mathbf{\mu}_\mathbf{x_*})}{\partial \mathbf{x_*}}^\top
\mathbf{\Sigma}_\mathbf{x_*}
\frac{\partial \mathbf{\mu}_\text{GP}(\mathbf{\mu}_\mathbf{x_*})}{\partial \mathbf{x_*}}}_\text{1st Order} 
\end{aligned}
$$

In [ ]:
mu = meanf(Xtest_noisy).squeeze()
var = varf(Xtest_noisy).squeeze()

In [ ]:
# demo sample
xtest_noisy = Xtest_noisy[demo_sample_idx]

# input covariance
input_cov = jnp.array([x_noise]).reshape(-1, 1)

# gradient predictive mean
f = lambda x: meanf(x).squeeze()

y_mu = f(xtest_noisy)

df = jax.grad(f)

# predictive mean
dy_mu = df(xtest_noisy)
chex.assert_equal_shape([dy_mu, xtest_noisy])

# predictive covariance
covar_to1 = dy_mu[None, :] @ input_cov @ dy_mu[:, None]
chex.assert_shape(
    covar_to1,
    (
        n_features,
        n_features,
    ),
)

# predictive variance
var_to1 = jnp.diag(covar_to1).squeeze()

one_stddev_to1 = 1.96 * jnp.sqrt(var_to1.squeeze()) + one_stddev

In [ ]:
mu_demo["taylor"] = y_mu

In [ ]:
# generate samples from the distribution
lin_samples = jax.random.multivariate_normal(
    key, jnp.atleast_1d(y_mu), covar_to1, (n_mc_points,)
)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
sns.kdeplot(ax=ax, y=y_mu_mc.ravel(), bw_adjust=3.0, linewidth=4, label="MC Samples")
sns.kdeplot(
    ax=ax, y=lin_samples.ravel(), bw_adjust=3.0, linewidth=4, label="Taylor Approx"
)
# sns.kdeplot(ax=ax, y=mm_samples.squeeze(), bw_adjust=3., linewidth=4, label='Moment Matching',)
ax.set(xlabel="", xticklabels="", ylim=(-2.1, 2.1), yticklabels="")
ax.axhline(mu_demo["standard"], color="black", linestyle="--", label="Mean Prediction")
plt.legend()
plt.show()

In [ ]:
def init_taylor_o1_transform(
    meanf: Callable, covarf: Callable, covariance: bool = False
) -> Callable:

    f = lambda x: meanf(x).squeeze()

    gradf = jax.grad(f)

    def apply_transform(x, x_cov):

        # ===================
        # Mean
        # ===================

        y_mu = f(x)
        y_mu = jnp.atleast_1d(y_mu)

        # ===================
        # Co/Variance
        # ===================
        y_cov = covarf(x)
        y_cov = jnp.atleast_2d(y_cov)

        # gradient of the
        dy_mu = gradf(x)

        # gradient
        x_cov = jnp.atleast_2d(x_cov)

        y_cov = dy_mu[None, :] @ x_cov @ dy_mu[:, None]

        if not covariance:
            y_cov = jnp.diag(y_cov).squeeze()
            y_cov = jnp.atleast_1d(y_cov)

        return y_mu, y_cov

    return apply_transform

In [ ]:
# init function
taylor_o1_transform = init_taylor_o1_transform(meanf, covarf, False)

# apply function
xtest_noisy = Xtest_noisy[demo_sample_idx]
input_cov = jnp.array([x_noise]).reshape(-1, 1)

y_mu, y_var = taylor_o1_transform(xtest_noisy, input_cov)

chex.assert_equal_shape([y_mu, y_var])
chex.assert_shape(y_mu, (n_features,))

In [ ]:
taylor_o1_transform_vectorized = jax.jit(
    jax.vmap(taylor_o1_transform, in_axes=(0, None))
)

In [ ]:
y_mu, y_var_o1 = taylor_o1_transform_vectorized(Xtest_noisy, input_cov)


chex.assert_equal_shape([y_mu, y_var_o1])
chex.assert_shape(
    y_mu,
    (
        ntest,
        n_features,
    ),
)


# y_std = 1.96 * jnp.sqrt(y_var)

In [ ]:
plot_1D_GP_noisy(Xtest_noisy, y_mu, y_var_o1);

### Linearization (2nd Order)

Here we do the same Taylor expansion except we can do the second order approximation.

$$
\begin{aligned}
\tilde{\mathbf{\mu}}_\text{LinGP}(\mathbf{x_*})
&= \mathbf{\mu}_\text{GP}(\mathbf{\mu}_\mathbf{x_*}) +
\underbrace{\frac{1}{2} \text{Tr}\left\{ \frac{\partial^2 \mathbf{\mu}_\text{GP}(\mathbf{\mu}_\mathbf{x_*})}{\partial \mathbf{x_*} \partial \mathbf{x_*}^\top}  \mathbf{\Sigma}_\mathbf{x_*}\right\}}_\text{second Order} \\
\tilde{\mathbf{\Sigma}}^2_\text{LinGP} (\mathbf{x_*}) &= 
\mathbf{\Sigma}^2_\text{GP}(\mathbf{\mu}_\mathbf{x_*}) + 
\underbrace{\frac{\partial \mathbf{\mu}_\text{GP}(\mathbf{\mu}_\mathbf{x_*})}{\partial \mathbf{x_*}}^\top
\mathbf{\Sigma}_\mathbf{x_*}
\frac{\partial \mathbf{\mu}_\text{GP}(\mathbf{\mu}_\mathbf{x_*})}{\partial \mathbf{x_*}}}_\text{1st Order} +
\underbrace{\frac{1}{2} \text{Tr}\left\{ \frac{\partial^2 \mathbf{\Sigma}^2_\text{GP}(\mathbf{\mu}_\mathbf{x_*})}{\partial \mathbf{x_*} \partial \mathbf{x_*}^\top}  \mathbf{\Sigma}_\mathbf{x_*}\right\}}_\text{second Order}
\end{aligned}
$$

**Mean Function**

In [ ]:
# mean function
f = lambda x: meanf(x).squeeze()

y_mu = f(xtest_noisy)
y_mu = jnp.atleast_1d(y_mu)

chex.assert_shape(y_mu, (n_features,))

# correction
# 2nd derivative of mean function
d2f = lambda x: jnp.atleast_2d(jax.hessian(f)(x).squeeze())

d2y_mu = d2f(xtest_noisy)

chex.assert_shape(d2y_mu, (n_features, n_features))

term_o2 = 0.5 * jnp.trace(d2y_mu @ input_cov)

y_mu += term_o2

chex.assert_equal_shape([y_mu, xtest_noisy])

In [ ]:
mu_demo["taylor_o2"] = y_mu

**Covariance Function**

In [ ]:
# 0th Order Term
vf = lambda x: jnp.atleast_1d(varf(x).squeeze())
cvf = lambda x: jnp.atleast_2d(covarf(x).squeeze())

y_cov = cvf(xtest_noisy)

chex.assert_shape(y_cov, (n_features, n_features))

# 1st Order Term

# 1st derivative of mean function
df = lambda x: jnp.atleast_1d(jax.grad(f)(x).squeeze())
dy_mu = df(xtest_noisy)

chex.assert_equal_shape([dy_mu, xtest_noisy])

# covariance
term_o1 = dy_mu[None, :] @ x_cov @ dy_mu[:, None].T

chex.assert_shape(term_o1, (n_features, n_features))

# 2nd Order Term

# correction
# 2nd derivative of variance function
vf = lambda x: varf(x).squeeze()
dv2f = jax.hessian(vf)

d2y_var = dv2f(xtest_noisy)

chex.assert_shape(d2y_var, (n_features, n_features))


term_o2 = 0.5 * jnp.trace(d2y_var @ input_cov)


y_cov += term_o1 + term_o2

chex.assert_shape(y_cov, (n_features, n_features))

y_var = jnp.diag(y_cov)

In [ ]:
# mu_demo["taylor_o2"] = y_mu

In [ ]:
# generate samples from the distribution
lin_o2_samples = jax.random.multivariate_normal(
    key, jnp.atleast_1d(y_mu), y_cov, (n_mc_points,)
)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
sns.kdeplot(ax=ax, y=y_mu_mc.ravel(), bw_adjust=3.0, linewidth=4, label="MC Samples")
sns.kdeplot(
    ax=ax,
    y=lin_samples.ravel(),
    bw_adjust=3.0,
    linewidth=4,
    label=r"Taylor Approx ($\mathcal{O}(1)$)",
)
sns.kdeplot(
    ax=ax,
    y=lin_o2_samples.ravel(),
    bw_adjust=3.0,
    linewidth=4,
    label=r"Taylor Approx ($\mathcal{O}(2)$)",
)
# sns.kdeplot(ax=ax, y=mm_samples.squeeze(), bw_adjust=3., linewidth=4, label='Moment Matching',)
ax.set(xlabel="", xticklabels="", ylim=(-2.1, 2.1), yticklabels="")
ax.axhline(mu_demo["standard"], color="black", linestyle="--", label="Mean Prediction")
plt.legend()
plt.show()

In [ ]:
def init_taylor_o2_transform(
    meanf: Callable, covarf: Callable, covariance: bool = False
) -> Callable:

    # mean function
    f = lambda x: meanf(x).squeeze()
    # derivative of Mean function
    df = lambda x: jnp.atleast_1d(jax.grad(f)(x).squeeze())
    # 2nd derivative of mean function
    d2f = lambda x: jnp.atleast_2d(jax.hessian(f)(x).squeeze())
    # 2nd derivative of variance function
    cvf = lambda x: jnp.diag(covarf(x)).squeeze()
    dv2f = jax.hessian(vf)

    def apply_transform(x, x_cov):

        # ===================
        # Mean
        # ===================

        y_mu = f(x)
        y_mu = jnp.atleast_1d(y_mu)

        # apply 2nd order correction
        d2y_mu = d2f(x)

        y_mu += 0.5 * jnp.trace(d2y_mu @ x_cov)

        # ===================
        # Co/Variance
        # ===================
        y_cov = covarf(x)

        # 1st order correction
        # gradient of the
        dy_mu = df(x)

        # gradient
        x_cov = jnp.atleast_2d(x_cov)

        y_cov += dy_mu[None, :] @ x_cov @ dy_mu[:, None]

        # 2nd order correction
        d2y_var = dv2f(x)
        y_cov += 0.5 * jnp.trace(d2y_var @ x_cov)

        if not covariance:
            y_cov = jnp.diag(y_cov).squeeze()
            y_cov = jnp.atleast_1d(y_cov)

        return y_mu, y_cov

    return apply_transform

In [ ]:
# init function
taylor_o2_transform = init_taylor_o2_transform(meanf, varf, False)

# apply function
xtest_noisy = Xtest_noisy[demo_sample_idx]
input_cov = jnp.array([x_noise]).reshape(-1, 1)

y_mu, y_var = taylor_o2_transform(xtest_noisy, input_cov)

chex.assert_equal_shape([y_mu, y_var])
chex.assert_shape(y_mu, (n_features,))

In [ ]:
taylor_o2_transform_vectorized = jax.jit(
    jax.vmap(taylor_o2_transform, in_axes=(0, None))
)

In [ ]:
y_mu, y_var_o2 = taylor_o2_transform_vectorized(Xtest_noisy, input_cov)


chex.assert_equal_shape([y_mu, y_var_o2])
chex.assert_shape(
    y_mu,
    (
        ntest,
        n_features,
    ),
)

# y_var = var.squeeze() + y_var_o2.squeeze()

In [ ]:
plot_1D_GP_noisy(Xtest_noisy, y_mu, y_var_o2);

## Unscented Transformation

#### Unscented Weights

In [ ]:
from typing import Optional, Tuple


def get_unscented_weights(
    n_features: int,
    kappa: Optional[float] = None,
    alpha: float = 1.0,
    beta: float = 2.0,
) -> Tuple[float, float]:
    """Generate normalizers for MCMC samples"""

    # calculate kappa value
    if kappa is None:
        kappa = jnp.maximum(3.0 - n_features, 0.0)

    lam = alpha**2 * (n_features + kappa) - n_features
    wm = 1.0 / (2.0 * (n_features + lam)) * np.ones(2 * n_features + 1)
    wc = wm.copy()
    wm = jax.ops.index_update(wm, 0, lam / (n_features + lam))
    wc = jax.ops.index_update(wc, 0, wm[0] + (1 - alpha**2 + beta))
    return wm, wc

In [ ]:
n_features = 1
alpha = 1.0
beta = 2.0
kappa = None
n_sigma_points = 3

wm, wc = get_unscented_weights(n_features, kappa, alpha, beta)

chex.assert_equal_shape([wm, wc])
chex.assert_shape(wm, (n_sigma_points,))

# Wm, Wc = jnp.diag(wm), jnp.diag(wc)

# chex.assert_equal_shape([Wm, Wc])
# chex.assert_shape(Wm, (n_sigma_points,n_sigma_points))

Wm, Wc = wm, jnp.diag(wc)

chex.assert_shape(Wc, (n_sigma_points, n_sigma_points))
chex.assert_shape(Wm, (n_sigma_points,))

#### Sigma Points

In [ ]:
def get_unscented_sigma_points(
    n_features: int, kappa: Optional[float] = None, alpha: float = 1.0
) -> Tuple[chex.Array, chex.Array]:
    """Generate Unscented samples"""

    # calculate kappa value
    if kappa is None:
        kappa = np.maximum(3.0 - n_features, 0.0)

    lam = alpha**2 * (n_features + kappa) - n_features
    c = np.sqrt(n_features + lam)
    return np.hstack(
        (np.zeros((n_features, 1)), c * np.eye(n_features), -c * np.eye(n_features))
    )

In [ ]:
kappa = None
alpha = 1.0
n_features = 1
n_sigma_points = 3


# generate sigma points
sigma_pts = get_unscented_sigma_points(n_features, kappa, alpha)

In [ ]:
# input covariance
L = jnp.linalg.cholesky(input_cov)

chex.assert_shape(
    L,
    (
        n_features,
        n_features,
    ),
)

# calculate sigma points
x_sigma_samples = xtest_noisy + L @ sigma_pts

chex.assert_shape(x_sigma_samples, (n_features, n_sigma_points))

# mean function predictions

f = lambda x: jnp.atleast_1d(meanf(x).squeeze())

y_mu_unc = jax.vmap(f, in_axes=2)(x_sigma_samples[:, None, :])

y_mu_unc = jnp.atleast_1d(y_mu_unc)

chex.assert_shape(
    y_mu_unc,
    (
        n_sigma_points,
        n_features,
    ),
)

# calculate weights of unscented points
y_mu = y_mu_unc.T @ Wm


chex.assert_shape(y_mu, (n_features,))

# calculate covariance of MC samples
dfx = y_mu_unc - y_mu

chex.assert_shape(
    dfx,
    (
        n_sigma_points,
        n_features,
    ),
)

y_cov = dfx.T @ Wc @ dfx

chex.assert_shape(
    y_cov,
    (
        n_features,
        n_features,
    ),
)

# input-output covariance
y_covx = wc * (x_sigma_samples - xtest_noisy) @ dfx

chex.assert_shape(
    y_cov,
    (
        n_features,
        n_features,
    ),
)

In [ ]:
mu_demo["sigma"] = y_mu

In [ ]:
# plt.hist(x_mc_samples.squeeze(), bins=100, density=True);
fig, ax = plt.subplots(figsize=(7, 1))
sns.kdeplot(
    ax=ax,
    x=x_mc_samples.squeeze(),
    bw_adjust=3.0,
    color="black",
    linewidth=4,
    fill=True,
    label=r"$p(x)$",
)
ax.axvline(xtest_noisy, linestyle="--", color="black", linewidth=4)
ax.axvline(sigma_pts[:, 0], linestyle="--", color="red", linewidth=4)
ax.axvline(sigma_pts[:, 1], linestyle="--", color="red", linewidth=2)
ax.axvline(sigma_pts[:, 2], linestyle="--", color="red", linewidth=2)
ax.set(yticklabels="", ylabel="", xticklabels="", xlim=(-10, 10))
# ax.set_ylim(-1.5, 1.1)
plt.legend(fontsize=15)
plt.show()

In [ ]:
# generate samples from the distribution
unscented_samples = jax.random.multivariate_normal(
    key, jnp.atleast_1d(y_mu), y_cov, (n_mc_points,)
)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
sns.kdeplot(
    ax=ax,
    y=unscented_samples.ravel(),
    bw_adjust=3.0,
    linewidth=4,
    color="red",
    label="Unscented",
)
ax.set(xlabel="", xticklabels="", ylim=(-2.1, 2.1), yticklabels="")
ax.axhline(y_mu_unc[0], linestyle="--", color="red", linewidth=4)
ax.axhline(y_mu_unc[1], linestyle="--", color="red", linewidth=4)
ax.axhline(y_mu_unc[2], linestyle="--", color="red", linewidth=4)
ax.axhline(mu_demo["standard"], color="black", linestyle="--", label="Mean Prediction")
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
sns.kdeplot(ax=ax, y=y_mu_mc.ravel(), bw_adjust=3.0, linewidth=4, label="MC Samples")
sns.kdeplot(
    ax=ax,
    y=lin_samples.ravel(),
    bw_adjust=3.0,
    linewidth=4,
    label=r"Taylor Approx ($\mathcal{O}(1)$)",
)
sns.kdeplot(
    ax=ax,
    y=lin_o2_samples.ravel(),
    bw_adjust=3.0,
    linewidth=4,
    label=r"Taylor Approx ($\mathcal{O}(2)$)",
)
sns.kdeplot(
    ax=ax,
    y=unscented_samples.ravel(),
    bw_adjust=3.0,
    linewidth=4,
    color="red",
    label="Unscented",
)
ax.set(xlabel="", xticklabels="", ylim=(-2.1, 2.1), yticklabels="")
ax.axhline(mu_demo["standard"], color="black", linestyle="--", label="Mean Prediction")
plt.legend()
plt.show()

### Vectorized Demo

In [ ]:
from typing import Callable


def init_unscented_transform(
    meanf: Callable,
    n_features: int,
    alpha: float = 1.0,
    beta: float = 2.0,
    kappa: Optional[float] = None,
    covariance: bool = False,
) -> Callable:

    f = lambda x: jnp.atleast_1d(meanf(x).squeeze())

    # get weights
    wm, wc = get_unscented_weights(n_features, kappa, alpha, beta)

    Wm, Wc = wm, jnp.diag(wc)

    # generate sigma points
    sigma_pts = get_unscented_sigma_points(n_features, kappa, alpha)

    def apply_transform(x, x_cov):

        # cholesky decomposition
        L = jnp.linalg.cholesky(x_cov)

        # calculate sigma points
        x_sigma_samples = x + L @ sigma_pts

        # propagate samples through function
        y_mu_unc = jax.vmap(f, in_axes=2)(x_sigma_samples[:, None, :])

        y_mu_unc = jnp.atleast_1d(y_mu_unc)

        # ===================
        # Mean
        # ===================

        # calculate weights of unscented points
        y_mu = y_mu_unc.T @ Wm

        # ===================
        # Covariance
        # ===================

        # calculate covariance of MC samples
        dfx = y_mu_unc - y_mu

        y_cov = dfx.T @ Wc @ dfx

        if not covariance:
            y_cov = jnp.diag(y_cov)

            y_cov = jnp.atleast_1d(y_cov)

        return y_mu, y_cov

    return apply_transform

In [ ]:
# init function
unscented_transform = init_unscented_transform(meanf, 1)

# apply function
xtest_noisy = Xtest_noisy[demo_sample_idx]
input_cov = jnp.array([x_noise]).reshape(-1, 1)

y_mu, y_var = unscented_transform(xtest_noisy, input_cov)

chex.assert_equal_shape([y_mu, y_var])
chex.assert_shape(y_mu, (n_features,))

In [ ]:
unscented_transform_vectorized = jax.jit(
    jax.vmap(unscented_transform, in_axes=(0, None))
)

In [ ]:
y_mu, y_var = unscented_transform_vectorized(Xtest_noisy, input_cov)

chex.assert_equal_shape([y_mu, y_var])
chex.assert_shape(
    y_mu,
    (
        ntest,
        n_features,
    ),
)

In [ ]:
# y_var = var.squeeze() + y_var_o2.squeeze()

plot_1D_GP_noisy(Xtest_noisy, y_mu, y_var);

In [ ]:
from jaxkern.gp.uncertain.linear import TaylorFirstOrder

In [ ]:
# calculate the mean
lin_mu = f(x_mu[:, None])

# calculate the covariance
lin_cov = df(x_mu[:, None]) @ x_cov @ df(x_mu[:, None]).T

In [ ]:
print(f"Mean: {lin_mu}")
print(f"Covariance: {lin_cov}")

In [ ]:
# generate samples from the distribution
lin_samples = jax.random.multivariate_normal(key, lin_mu, lin_cov, (n_mc_points,))

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
sns.kdeplot(ax=ax, y=y_mc_samples, bw_adjust=3.0, linewidth=4, label="MC Samples")
sns.kdeplot(
    ax=ax, y=lin_samples.squeeze(), bw_adjust=3.0, linewidth=4, label="Taylor Approx"
)
# sns.kdeplot(ax=ax, y=mm_samples.squeeze(), bw_adjust=3., linewidth=4, label='Moment Matching',)
ax.set(xlabel="", xticklabels="", ylim=(-2.1, 2.1), yticklabels="")
plt.legend()
plt.show()

### Moment Matching Transformation

So explicitly, we need to take expectations (integrals) of the GP predictive mean and variance w.r.t. our distribution for $\mathbf{x}_*$. 

$$
\begin{aligned}
\tilde{\mu}_{GP}(\mathbf{x_*}) &= \int \mu_{GP}(\mathbf{x_*}) p(\mathbf{x_*})d\mathbf{x_*} \\
\tilde{\sigma}^2_{GP}(\mathbf{x}_*) &= \int \sigma^2_{GP}(\mathbf{x_*}) p(\mathbf{x_*}) d\mathbf{x}_* + \int  \mu_{GP}^2(\mathbf{x_*})p(\mathbf{x_*})d\mathbf{x_*}  - \left[ \int \mu_{GP}(\mathbf{x_*}) p(\mathbf{x_*})d\mathbf{x_*}\right]^2
\end{aligned}
$$

After some manipulation, this results in the follow equations for the predictive mean and variance (**cite**).


$$
\begin{aligned}
\tilde{\mu}_{GP}(\mathbf{x_*}) &= \Psi_1^\top\alpha \\
\tilde{\sigma}^2_{GP}(\mathbf{x}_*)
&=
\psi_0 - \text{Tr}\left( \left(\mathbf{K}_{GP}^{-1}  - \alpha\alpha^\top\right) \Psi_2\right) - \text{Tr}\left( \Psi_1\Psi_1^\top\alpha\alpha^\top \right)
\end{aligned}
$$

where we have $\Psi_i$ quantities called kernel expectations denoted by:

$$
\begin{aligned}
[\Psi_0][\Psi_0]_{i}  &= \int k(\mathbf{x}_i, \mathbf{x}_i)p(\mathbf{x}_i)d\mathbf{x}_i \\
[\Psi_1]_{ij} &= \int k(\mathbf{x}_i, \mathbf{y}_i)p(\mathbf{x}_i)d\mathbf{x}_i \\
[\Psi_2]_{ijk} &= \int k(\mathbf{x}_i, \mathbf{y}_j)k(\mathbf{x}_i, \mathbf{z}_k) d\mathbf{x}_i
\end{aligned}
$$

Notice how we have expectations where we need to use quadrature methods to calculate these quantities. In this example, we will use the GaussHermite method.


In [ ]:
from jaxkern.gp.uncertain.moment import (
    MomentMatchingTransform,
    GaussHermite,
    get_quadrature_sigma_points,
)

In [ ]:
degree = 20

# initialize model
mm_clf = MomentMatchingTransform(model, GaussHermite, degree=degree, jitted=False)

In [ ]:
# calculate kernel expectations
psi0 = mm_clf.kernel_expectations.expectation_xkx(x_mu[:, None], x_cov[None, :])
psi1 = mm_clf.kernel_expectations.expectation_xkxy(
    x_mu[:, None], x_cov[None, :], mm_clf.model.X_train_
)
psi2 = mm_clf.kernel_expectations.expectation_xkxyz(
    x_mu[:, None], x_cov[None, :], mm_clf.model.X_train_, mm_clf.model.X_train_
)

# calculate mean function
mm_mu = psi1 @ mm_clf.model.weights

# calculate covariance function
t1 = psi0
t2 = psi2 @ (mm_clf.K_inv_ - mm_clf.model.weights @ mm_clf.model.weights.T)
t3 = psi1.T @ psi1 @ mm_clf.model.weights @ mm_clf.model.weights.T

mm_cov = t1 - np.trace(t2, axis1=1, axis2=2) - np.trace(mm_mu**2)
mm_cov = mm_cov.reshape(-1, 1)

In [ ]:
print(f"Mean: {mm_mu}")
print(f"Covariance: {mm_cov}")

In [ ]:
# generate samples from distribution
mm_samples = jax.random.multivariate_normal(key, mm_mu, mm_cov, (n_mc_points,))

In [ ]:
fig, ax = plt.subplots(figsize=(3, 5))
sns.kdeplot(ax=ax, y=y_mc_samples, bw_adjust=3.0, linewidth=4, label="MC Samples")
sns.kdeplot(
    ax=ax, y=lin_samples.squeeze(), bw_adjust=3.0, linewidth=4, label="Taylor Approx"
)
sns.kdeplot(
    ax=ax,
    y=mm_samples.squeeze(),
    bw_adjust=3.0,
    linewidth=4,
    label="Moment Matching",
)
# ax.set_ylim(-1.5, 1.1)
ax.set(xlabel="", xticklabels="", ylim=(-2.1, 2.1), yticklabels="")
# ax.set_xticklabels("")
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
sns.kdeplot(ax=ax, x=y_mc_samples, bw_adjust=3.0, linewidth=4, label="MC Samples")

sns.kdeplot(
    ax=ax,
    x=lin_samples.squeeze(),
    bw_adjust=3.0,
    linewidth=4,
    label="Taylor Approx",
    color="orange",
)
ax.axvline(
    lin_mu, linestyle="--", label="Mean (Taylor Approx.)", linewidth=2, color="orange"
)
sns.kdeplot(
    ax=ax,
    x=mm_samples.squeeze(),
    bw_adjust=3.0,
    linewidth=4,
    label="Moment Matching",
    color="green",
)
ax.axvline(
    mm_mu, linestyle="--", label="Mean (Moment Matching)", linewidth=2, color="green"
)
# ax.set_ylim(-1.5, 1.1)
ax.set(xlabel="", xticklabels="", yticklabels="", ylim=(0, 2.0), ylabel="")
# ax.set_xticklabels("")
plt.legend(fontsize=12)
plt.show()